In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools  import tool
from crewai import Crew
from crewai import Task
from crewai import Agent

In [2]:
# Loading the LLM
from langchain_ollama import ChatOllama
llm = ChatOllama(model= "mistral:instruct")

### Getting PDF- docs and stroing in ChromaDB vector DB

In [4]:
rag_tool = PDFSearchTool(pdf= r'D:\Company_Assignments\Attack Capital\NLP\Multi-Agent-Application\transformer.pdf',
    config=dict(
        llm=dict(
            provider="ollama", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="mistral:instruct",
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

c:\Users\Chaitanya\anaconda3\envs\genai\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\Chaitanya\anaconda3\envs\genai\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\Chaitanya\anaconda3\envs\genai\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not 

In [5]:
rag_tool.run("What is finetuning?")

Using Tool: Search a PDF's content


'Relevant Content:\nof fine-tuning LLMs\n\nLLMsaretrainedusingaprocesscalledunsupervisedlearning.Thisinvolvesfeedingthemodelmassiveamountsoftextdata,suchasbooks,articles,andwebsites,andhavingthemodellearnthepatternsandrelationshipsbetweenwordsandphrasesinthetext.Themodelisthenfine-tunedonaspecifictask,suchaslanguagetranslationortextsummarization. Fine-TuningFine-tuningistheprocessoftrainingapre-trainedlargelanguagemodelonaspecifictaskusingasmallerdataset.Thisallowsthemodeltolearntask-specificfeaturesandimproveitsperformance.Thefine-tuningprocesstypicallyinvolvesfreezingtheweightsofthepre-trainedmodelandonlytrainingthetask-specificlayers.Whenfine-tuningamodel,it\'simportanttoconsiderfactorssuchasthesizeofthefine-tuningdataset,thechoiceofoptimizerandlearningrate,andthechoiceofevaluationmetrics\n\nPreprocessingTextnormalizationistheprocessofconvertingtexttoastandardformat,suchaslowercasingalltext,removingspecialcharacters,andconvertingnumberstotheirwrittenform.Tokenizationistheprocessofbr

### Trying out the same search with creawai

In [6]:

os.environ['TAVILY_API_KEY'] = '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'
web_search_tool = TavilySearchResults(k=3)

In [8]:
web_search_tool.run("What is the finetuning in LLMs?")

[{'url': 'https://www.lakera.ai/blog/llm-fine-tuning-guide',
  'content': '**\nTools & best practices for fine-tuning LLMs\nChallenges & limitations of LLM fine tuning\nSome of the main challenges and limitations associated with fine-tuning LLMs:\nUnderstanding fine tuning LLMs: What is LLM Fine-Tuning\nModel fine tuning is a process where a pre-trained model, which has already learned some patterns and features on a large dataset, is further trained (or "fine tuned") on a smaller, domain-specific dataset. How to choose the best pre-trained model for fine-tuning\nChoosing the most suitable pre-trained language model (LLM) for fine-tuning is crucial in natural language processing tasks. It also guided the reader on choosing the best pre-trained model for fine-tuning and emphasized the importance of security measures, including tools like Lakera, to protect LLMs and applications from threats.\n The Ultimate Guide to LLM Fine Tuning: Best Practices & Tools\nWhat is model fine tuning and h

# Creating A Main -Workflow

In [9]:
@tool
def router_tool(question):
  """Router Function"""
  if 'transformer' in question:
    return 'vectorstore'
  else:
    return 'web_search'

In [10]:
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [11]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [12]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [13]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [14]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [15]:
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [16]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
)

In [17]:
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [18]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

In [19]:
# Creating a Workflow
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,
)

In [22]:
# Passing an Input to get an answer - providing a relevant question
inputs ={"question":"Do transformer use Attention mechanism ?"}
result = rag_crew.kickoff(inputs=inputs)

 [2024-08-08 23:30:44][DEBUG]: == Working Agent: Router
 [2024-08-08 23:30:44][INFO]: == Starting Task: Analyse the keywords in the question Do transformer use Attention mechanism ?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
 Action: router_tool
   Action Input: {"question": "Do transformer use Attention mechanism"}
   Observation: vectorstore 

I encountered an error while trying to use the tool. This was the error: router_tool() missing 1 required positional argument: 'question'.
 Tool router_tool accepts these inputs: router_tool() - Router Function 

 Thought: To determine if this question is suitable for vectorstore search or web search, I need to consider its specificity and complexity. Since the qu

In [23]:

print(result)

Yes, Transformers do indeed utilize an Attention Mechanism. The Attention Mechanism helps the model to focus on certain parts of the input when generating its output, making it particularly useful in tasks such as translation and summarization. This mechanism was introduced in the paper "Attention is All You Need" by Vaswani et al., published in 2017.
